NETTOYAGE / FILTRAGE

In [ ]:
# Name Basics

'''
Colonnes à supprimer : Aucune
Lignes à supprimer:
    - knownfortitle = /N
    - primaryprofession = /N
'''

In [ ]:
# Title Akas

'''
Colonnes à supprimer : 
Lignes à supprimer: 
    - Faire un groupby par ID de film
    - A voir plus tard pour filtrer avec un contains sur language et/ou région
'''

In [ ]:
# Title Basics

'''
Colonnes à supprimer : 
Lignes à supprimer: 
    - Titletype = movie
    - A voir pour filtrer par années par la suite
    - A voir pour filtrer selon durée des films (trop courts ou trop longs)
    - isAdult = 1 et genre = adult (uniquement)
    - Vérifier classification de Basic instinct

'''

In [ ]:
# Title Crew

'''
RAS
'''

In [ ]:
# Title Episodes

'''
A voir : concatener numberEpisode et numberSeason
'''

In [ ]:
# Title Principals

'''
Colonnes à supprimer : job, character
'''

In [ ]:
# Title Ratings

'''
RAS
'''

In [ ]:
# TMDB Full

'''
Colonnes à supprimer : 
Lignes à supprimer: 
'''